In [1]:
import pandas as pd
import os

In [2]:
results = pd.read_csv("results.csv", index_col=0, sep="\t")
finetuned_results = pd.read_csv("finetuned_results.csv", index_col=0, sep="\t")

In [3]:
def format_names_for_macros(latex_str):
    return (latex_str
            .replace("SimCSE25", "\simcseuno{}")
            .replace("SimCSE21", "\simcsedue{}")
            .replace("GreBerta", "\greberta{}")
            .replace("LaBerta", "\laberta{}")
            .replace("SPhilBERTa", "\sphilberta{}")
            .replace("PhilBERTa", "\philberta{}")
    )

In [4]:
# make table good looking in latex
t_results = results.T.copy()
for idx, i in enumerate(["lang", "split", "setting", "author", "model"]):
    newcol = []
    for j in t_results.index.str.split("_"):
        newcol.append(j[idx])
    t_results[i] = newcol
t_results = t_results.reset_index(drop=True).set_index(["lang", "split", "setting", "model"])
t_results = t_results.drop("author", axis=1)
t_results.columns = ["\\rotatebox{{90}}{{{col_name}}}".format(col_name=col.replace("_", "-")) for col in t_results.columns]
t_results = t_results.loc[:, ~t_results.columns.str.contains("num")]
col_format = "l" * 4 + "c" * (len(t_results.columns))
col_format = col_format[:-18] + "|" + col_format[-18:-9] + "|" + col_format[-9:]  # Add vertical line before the last 8 columns
formatters = {
        **{
            i: lambda x: f"{x:.2f}" for i in t_results.columns if "P@" not in i and "NDCG" not in i
            },
        **{
            i: lambda x: "\gradient{" + f"{x:.2f}" + "}"
            for i in t_results.columns if "P@" in i or "map" in i or "recip-rank" in i
            or "bpref" in i or "Rprec" in i
            },
        **{
            i: lambda x: r"\newgradient{" + f"{x:.2f}" + "}"
            for i in t_results.columns if "NDCG" in i
            },
    }

def save_to_latex(results, file_name, col_format, formatters):
    latex_str = results.to_latex(
        index=True,
        float_format="%.3f",
        column_format=col_format,
        formatters=formatters,
    )
    latex_str = (latex_str.replace(r'\toprule', '').replace("cline", "cmidrule"))
    lines = latex_str.split("\n")
    new_lines = []
    for idx, line in enumerate(lines[:-1]):
        if "rule" in line:
            if "rule" in lines[idx + 1]:
                continue
        if len(line.split("\cmidrule")) > 1:
            line = "\cmidrule" + line.split('\cmidrule')[1]
            line = line.strip()
        new_lines.append(line)
    latex_str = "\n".join(new_lines)

    latex_str = format_names_for_macros(latex_str)

    os.makedirs("tables", exist_ok=True)
    with open(f"tables/{file_name}", "w") as f:
        f.write(latex_str)

save_to_latex(t_results, "results.tex", col_format, formatters)

In [5]:
col_to_keep = ["{map", "recip-rank", "P@5}", "P@10}", "NDCG@5}", "NDCG@10}"]
col_to_keep = [i for i in t_results.columns if any(j in i for j in col_to_keep)]
lang_col_format = "l" * 3 + "c" * (len(col_to_keep))
latin_pretrained_results = t_results.loc[("latin"), col_to_keep]
save_to_latex(latin_pretrained_results, "latin_pretrained_results.tex", lang_col_format, formatters)
greek_pretrained_results = t_results.loc["greek", col_to_keep]
save_to_latex(greek_pretrained_results, "greek_pretrained_results.tex", lang_col_format, formatters)
latin_greek_pretrained_results = t_results.loc[("latin+greek"), col_to_keep]
save_to_latex(latin_greek_pretrained_results, "latin_greek_pretrained_results.tex", lang_col_format, formatters)
greek_latin_pretrained_results = t_results.loc[("greek+latin"), col_to_keep]
save_to_latex(greek_latin_pretrained_results, "greek_latin_pretrained_results.tex", lang_col_format, formatters)
multilingual_pretrained_results = pd.concat([t_results.loc[("greek"), col_to_keep], t_results.loc[("latin"), col_to_keep]], axis=1)
save_to_latex(multilingual_pretrained_results, "multilingual_pretrained_results.tex", lang_col_format + lang_col_format[3:], formatters)
crosslingual_pretrained_results = pd.concat([t_results.loc[("latin+greek"), col_to_keep], t_results.loc[("greek+latin"), col_to_keep]], axis=1)
save_to_latex(crosslingual_pretrained_results, "crosslingual_pretrained_results.tex", lang_col_format + lang_col_format[3:], formatters)

In [6]:
t_finetuned_results = finetuned_results.T.copy()
for idx, i in enumerate(["lang", "split", "setting", "author", "model"]):
    newcol = []
    for j in t_finetuned_results.index.str.split("_"):
        newcol.append(j[idx])
    t_finetuned_results[i] = newcol
t_finetuned_results = t_finetuned_results.reset_index(drop=True).set_index(["lang", "split", "setting", "model"])
t_finetuned_results = t_finetuned_results.drop("author", axis=1)
t_finetuned_results.columns = ["\\rotatebox{{90}}{{{col_name}}}".format(col_name=col.replace("_", "-")) for col in t_finetuned_results.columns]

t_finetuned_results = t_finetuned_results.loc[:, ~t_finetuned_results.columns.str.contains("num")]
col_format = "l" * 4 + "c" * (len(t_finetuned_results.columns))
col_format = col_format[:-18] + "|" + col_format[-18:-9] + "|" + col_format[-9:]  # Add vertical line before the last 8 columns
save_to_latex(t_finetuned_results, "finetuned_results.tex", col_format=col_format, formatters=formatters)
# t_finetuned_results

In [7]:
save_to_latex(
    t_finetuned_results.loc[:, col_to_keep],
    "selected_finetuned_results.tex",
    col_format="l" * 4 + "c" * len(col_to_keep),
    formatters=formatters
)

In [8]:
benchmark_finetuned_results = (
    pd.concat([t_finetuned_results, t_results])
    .reset_index())
benchmark_finetuned_results = benchmark_finetuned_results.loc[
    benchmark_finetuned_results["model"].apply(lambda x: x in ["SPhilBERTa", "SimCSE25", "SimCSE21"])
].set_index(["lang", "split", "setting", "model"]).sort_index(ascending=False)

In [25]:
t_results.loc["latin+greek", "\\rotatebox{90}{map}"]

/tmp/ipykernel_3403634/1144147878.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  t_results.loc["latin+greek", "\\rotatebox{90}{map}"]


split   setting        model     
silver  target         GreBerta      0.076761
                       LaBerta       0.081383
                       PhilBERTa     0.513198
                       SPhilBERTa    0.928204
        target+random  GreBerta      0.058572
                       LaBerta       0.490596
                       PhilBERTa     0.957747
                       SPhilBERTa    0.903178
gold    target         GreBerta      0.240748
                       LaBerta       0.145098
                       PhilBERTa     0.440205
                       SPhilBERTa    0.753014
        target+random  GreBerta      0.144141
                       LaBerta       0.151594
                       PhilBERTa     0.607761
                       SPhilBERTa    0.714042
Name: \rotatebox{90}{map}, dtype: float64

In [23]:
benchmark_finetuned_results.loc["latin+greek", "\\rotatebox{90}{map}"]
# print("\\rotatebox{90}{P@5}")

/tmp/ipykernel_3403634/1712784978.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  benchmark_finetuned_results.loc["latin+greek", "\\rotatebox{90}{map}"]


split   setting        model     
silver  target+random  SimCSE25      0.942972
                       SimCSE21      0.942617
                       SPhilBERTa    0.903178
        target         SimCSE25      0.927215
                       SimCSE21      0.931312
                       SPhilBERTa    0.928204
gold    target+random  SimCSE25      0.725032
                       SimCSE21      0.851541
                       SPhilBERTa    0.714042
        target         SimCSE25      0.741883
                       SimCSE21      0.768847
                       SPhilBERTa    0.753014
Name: \rotatebox{90}{map}, dtype: float64

In [9]:
latin_finetuned_results = benchmark_finetuned_results.loc[("latin"), col_to_keep]
save_to_latex(latin_finetuned_results, "latin_finetuned_results.tex", lang_col_format, formatters)
greek_finetuned_results = benchmark_finetuned_results.loc["greek", col_to_keep]
save_to_latex(greek_finetuned_results, "greek_finetuned_results.tex", lang_col_format, formatters)
latin_greek_finetuned_results = benchmark_finetuned_results.loc[("latin+greek"), col_to_keep]
save_to_latex(latin_greek_finetuned_results, "latin_greek_finetuned_results.tex", lang_col_format, formatters)
greek_latin_finetuned_results = benchmark_finetuned_results.loc[("greek+latin"), col_to_keep]
save_to_latex(greek_latin_finetuned_results, "greek_latin_finetuned_results.tex", lang_col_format, formatters)
multilingual_finetuned_results = pd.concat(
    [
        benchmark_finetuned_results.loc[("latin+greek"), col_to_keep], 
        benchmark_finetuned_results.loc[("greek+latin"), col_to_keep]
     ],
    axis=1)
save_to_latex(multilingual_finetuned_results, "multilingual_finetuned_results.tex", lang_col_format + lang_col_format[3:], formatters)

In [12]:
arranged_finetuned_results = pd.concat([
    pd.concat([greek_finetuned_results, latin_finetuned_results], axis=1),
    pd.concat([latin_greek_finetuned_results, greek_latin_finetuned_results], axis=1),
], axis=0)
save_to_latex(arranged_finetuned_results, "arranged_finetuned_results.tex", lang_col_format + lang_col_format[3:], formatters)

In [11]:
t_results

\rotatebox{90}{map}  \
lang        split  setting       model                             
latin       silver target        GreBerta               0.687870   
                                 LaBerta                0.893462   
                                 PhilBERTa              0.932090   
                                 SPhilBERTa             0.954387   
                   target+random GreBerta               0.824218   
...                                                          ...   
greek+latin gold   target        SPhilBERTa             0.727876   
                   target+random GreBerta               0.010138   
                                 LaBerta                0.023282   
                                 PhilBERTa              0.149055   
                                 SPhilBERTa             0.579086   

                                             \rotatebox{90}{gm-map}  \
lang        split  setting       model                                
latin       silver target        GreBerta                  0.623790   
                                 LaBerta                   0.862516   
                                 PhilBERTa                 0.921851   
                                 SPhilBERTa                0.947895   
                   target+random GreBerta                  0.774044   
...                                                             ...   
greek+latin gold   target        SPhilBERTa                0.684286   
                   target+random GreBerta                  0.008755   
                                 LaBerta                   0.015476   
                                 PhilBERTa                 0.094126   
                                 SPhilBERTa                0.469290   

                                             \rotatebox{90}{bpref}  \
lang        split  setting       model                               
latin       silver target        GreBerta                   0.6400   
                                 LaBerta                    0.8692   
                                 PhilBERTa                  0.9152   
                                 SPhilBERTa                 0.9376   
                   target+random GreBerta                   0.7884   
...                                                            ...   
greek+latin gold   target        SPhilBERTa                 0.6520   
                   target+random GreBerta                   0.0000   
                                 LaBerta                    0.0000   
                                 PhilBERTa                  0.0680   
                                 SPhilBERTa                 0.5000   

                                             \rotatebox{90}{Rprec}  \
lang        split  setting       model                               
latin       silver target        GreBerta                    0.632   
                                 LaBerta                     0.864   
                                 PhilBERTa                   0.892   
                                 SPhilBERTa                  0.922   
                   target+random GreBerta                    0.764   
...                                                            ...   
greek+latin gold   target        SPhilBERTa                  0.640   
                   target+random GreBerta                    0.000   
                                 LaBerta                     0.000   
                                 PhilBERTa                   0.100   
                                 SPhilBERTa                  0.500   

                                             \rotatebox{90}{recip-rank}  \
lang        split  setting       model                                    
latin       silver target        GreBerta                      0.967833   
                                 LaBerta                       0.973333   
                                 PhilBERTa                     0.995000   
                                 SPhilB